In [ ]:
# Hate Speech Detection

In [ ]:
# Install essential packages
!pip install transformers datasets keras_tuner nltk praw asyncpraw wordcloud deep-translator gradio wandb --quiet
!pip install tensorflow tensorflow-text tf-keras --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 137.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ✅ Import libraries
import pandas as pd
import numpy as np
import random
import re
import nltk
import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer
from nltk.corpus import stopwords
from deep_translator import GoogleTranslator
import gradio as gr
import wandb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download NLTK stopwords
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

In [ ]:
# Init W&B
wandb.init(project="hate_speech_detection", mode="disabled")  # Disable online logging for Colab

In [ ]:
!pip install transformers datasets --quiet

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return ' '.join([word for word in text.lower().split() if word not in STOPWORDS])


# Back Translation (25%)
def back_translate(texts, fraction=0.25, max_length=4000):

    num_to_translate = int(len(texts) * fraction)  # Compute 25% of data
    indices = random.sample(range(len(texts)), num_to_translate)  # Select random indices
    translated_texts = texts[:]  # Copy original data
    for idx in indices:
        text = texts[idx]
        try:
          if len(text) > max_length:
              text = text[:max_length]  # Truncate long text
              translated = GoogleTranslator(source='auto', target='fr').translate(text)
              back_translated = GoogleTranslator(source='fr', target='en').translate(translated)
              translated_texts[idx] = back_translated
        except:
          pass
    return translated_texts


In [ ]:

# ✅ Imports
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ✅ Load and sample 1% of the dataset
df = pd.read_csv("drive/MyDrive/HateSpeechDatasetBalanced.csv")
df = df[["Content", "Label"]]
df = df.sample(frac=0.01, random_state=42)
df["Label"] = df["Label"].astype(int)
texts = df["Content"].tolist()
df["Content"] = back_translate(texts)
df["Content"] = df["Content"].apply(clean_text)

# ✅ Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.rename_column("Label", "labels")
test_dataset = test_dataset.rename_column("Label", "labels")

# ✅ Tokenization
model_name = "distilbert-base-uncased"
distilbert_tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    return distilbert_tokenizer(batch["Content"], padding=True, truncation=True, max_length=128)

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# ✅ Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# ✅ Hyperparameters
lr = 2e-5
batch_size = 16
epochs = 4
weight_decay = 0.01

training_args = TrainingArguments(
    output_dir="./distilbert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    logging_dir="./distilbert_logs"
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# ✅ Train
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.450252,0.793531,0.802372,0.749077,0.863830
2,0.486200,0.451310,0.792842,0.792271,0.771505,0.814184
3,0.314700,0.530917,0.794219,0.799463,0.758270,0.845390
4,0.314700,0.608401,0.780454,0.775510,0.769553,0.781560


TrainOutput(global_step=1452, training_loss=0.33859205640051976, metrics={'train_runtime': 311.6583, 'train_samples_per_second': 74.543, 'train_steps_per_second': 4.659, 'total_flos': 769370651394048.0, 'train_loss': 0.33859205640051976, 'epoch': 4.0})

In [ ]:
# ✅ Tokenizer and Preprocessing
model_name = "albert-base-v2"
albert_tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_albert(batch):
    return albert_tokenizer(batch["Content"], padding=True, truncation=True, max_length=128)

train_dataset_albert = train_dataset.map(preprocess_albert, batched=True)
test_dataset_albert = test_dataset.map(preprocess_albert, batched=True)

train_dataset_albert = train_dataset_albert.rename_column("labels", "label")
test_dataset_albert = test_dataset_albert.rename_column("labels", "label")

train_dataset_albert.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset_albert.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

model_albert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args_albert = TrainingArguments(
    output_dir="./albert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./albert_logs"
)

trainer_albert = Trainer(
    model=model_albert,
    args=training_args_albert,
    train_dataset=train_dataset_albert,
    eval_dataset=test_dataset_albert,
    compute_metrics=compute_metrics
)

trainer_albert.train()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Map:   0%|          | 0/5808 [00:00<?, ? examples/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.576265,0.683414,0.642857,0.710120,0.587234
2,0.618900,0.509782,0.747419,0.749488,0.722368,0.778723
3,0.489200,0.513627,0.761184,0.783260,0.699777,0.889362
4,0.489200,0.510460,0.762560,0.773770,0.719512,0.836879


TrainOutput(global_step=1452, training_loss=0.5065997835689997, metrics={'train_runtime': 560.0153, 'train_samples_per_second': 41.485, 'train_steps_per_second': 2.593, 'total_flos': 138800175022080.0, 'train_loss': 0.5065997835689997, 'epoch': 4.0})

In [ ]:
# ✅ Step 0: Install required packages
!pip install transformers datasets --quiet

# ✅ Step 1: Imports
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm

# ✅ Step 2: Load & sample dataset
df = pd.read_csv("drive/MyDrive/HateSpeechDatasetBalanced.csv")
df = df[["Content", "Label"]].sample(frac=0.01, random_state=42)
df["Label"] = df["Label"].astype(int)

df["Content"] = back_translate(texts)
df["Content"] = df["Content"].apply(clean_text)

# ✅ Step 3: Train/Test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# ✅ Step 4: Tokenizer
model_name = "unitary/unbiased-toxic-roberta"
hatebert_tokenizer = RobertaTokenizer.from_pretrained(model_name)

# ✅ Step 5: Custom Dataset
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# ✅ Step 6: Dataset and DataLoader
train_dataset = HateSpeechDataset(train_df["Content"].tolist(), train_df["Label"].tolist(), hatebert_tokenizer)
test_dataset = HateSpeechDataset(test_df["Content"].tolist(), test_df["Label"].tolist(), hatebert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# ✅ Step 7: Load model with head replacement
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True  # ✅ This solves the 16→2 mismatch
)
print("✅ HateBERT model loaded with binary classification head.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Step 8: Optimizer and Loss
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# ✅ Step 9: Training loop
num_epochs = 4
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1}: Avg Loss = {avg_loss:.4f}")

# ✅ Step 10: Evaluation
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

# ✅ Step 11: Metrics
acc = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

print(f"\n🎯 Test Accuracy: {acc:.4f}")
print(f"📊 Precision: {precision:.4f} | Recall: {recall:.4f} | F1 Score: {f1:.4f}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at unitary/unbiased-toxic-roberta and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([16, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([16]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ HateBERT model loaded with binary classification head.


Epoch 1: 100%|██████████| 726/726 [02:27<00:00,  4.94it/s]


✅ Epoch 1: Avg Loss = 0.5287


Epoch 2: 100%|██████████| 726/726 [02:25<00:00,  5.00it/s]


✅ Epoch 2: Avg Loss = 0.4249


Epoch 3: 100%|██████████| 726/726 [02:26<00:00,  4.96it/s]


✅ Epoch 3: Avg Loss = 0.3390


Epoch 4: 100%|██████████| 726/726 [02:25<00:00,  5.00it/s]


✅ Epoch 4: Avg Loss = 0.2583

🎯 Test Accuracy: 0.7701
📊 Precision: 0.7849 | Recall: 0.7248 | F1 Score: 0.7537


In [ ]:
# Predict function
def tokenize_and_predict(data, model, tokenizer):
    inputs = tokenizer(data["text"], return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.argmax(outputs.logits, axis=1).numpy()

In [ ]:

# ✅ Define models to save
model_names = {
    "hatebert": "GroNLP/hateBERT",
    "distilbert": "distilbert-base-uncased",
    "albert": "albert-base-v2"
}

# ✅ Save directory base
base_path = "/content/drive/MyDrive/models"

# ✅ Save function
def save_model(model_name, model_id):
    save_path = os.path.join(base_path, model_name)
    print(f"⬇️ Loading {model_name} from {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
    tokenizer.save_pretrained(save_path)
    model.save_pretrained(save_path)
    print(f"✅ {model_name} saved at {save_path}")

# ✅ Run save for all models
for name, hf_id in model_names.items():
    save_model(name, hf_id)


⬇️ Loading hatebert from GroNLP/hateBERT


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ hatebert saved at /content/drive/MyDrive/models/hatebert
⬇️ Loading distilbert from distilbert-base-uncased


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ distilbert saved at /content/drive/MyDrive/models/distilbert
⬇️ Loading albert from albert-base-v2


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ albert saved at /content/drive/MyDrive/models/albert


In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import praw

# ================= Model Setup =================

models_info = {
    "HateBERT": "drive/MyDrive/models/hatebert",
    "DistilBERT": "drive/MyDrive/models/distilbert",
    "ALBERT": "drive/MyDrive/models/albert"
}

models = {}
tokenizers = {}

for name, path in models_info.items():
    try:
        models[name] = AutoModelForSequenceClassification.from_pretrained(path)
        tokenizers[name] = AutoTokenizer.from_pretrained(path)
    except Exception as e:
        print(f"❌ Failed to load {name}: {str(e)}")

# ================= Prediction =================

def predict(text, model, tokenizer):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            prediction = torch.argmax(outputs.logits, dim=1).item()
        return "Hate Speech" if prediction == 1 else "Non-Hate Speech"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def analyze(text):
    results = {}
    try:
        for name in models:
            pred = predict(text, models[name], tokenizers[name])
            results[name] = pred
        votes = list(results.values())
        final = max(set(votes), key=votes.count)
        return final, results, ""
    except Exception as e:
        return "Error", {}, str(e)

# ================= Reddit Scraper =================

reddit = praw.Reddit(
    client_id="HA_-gJsAuSuQychYtu3oIA",
    client_secret="RfGxHKmTb8gdgv-FBVRr-61SnZodxA",
    user_agent="hatespeech_model_tool by u/Embarrassed-Class752"
)

discrimination_keywords = [
    "misogynist", "anti-feminist", "women", "feminist", "she deserved", "girl", "female",
    "black", "african", "anti-black", "slavery", "racial", "thug", "n-word", "monkey",
    "jew", "jews", "zionist", "antisemitic", "holocaust", "israel",
    "lgbt", "gay", "lesbian", "trans", "homophobic", "pride", "groomer",
    "immigrant", "foreigners", "go back", "minorities", "illegal", "ethnic",
    "hate them", "racist", "discrimination", "inferior", "purity", "cleansing"
]

def scrape_reddit():
    subreddit = reddit.subreddit("all")
    for post in subreddit.new(limit=200):
        content = f"{post.title} {post.selftext}".lower()
        if any(keyword in content for keyword in discrimination_keywords):
            return post.selftext.strip() or post.title.strip()
    return "No matching post found."

def scrape_and_predict():
    text = scrape_reddit()
    pred, preds, err = analyze(text)
    return text, pred, preds, err

# ================= Gradio Interface =================

with gr.Blocks() as interface:
    gr.Markdown("🧠 **Hate Speech Detection with BERT Ensemble**")
    with gr.Tabs():
        with gr.TabItem("Single Text"):
            input_text = gr.Textbox(lines=2, placeholder="Enter Text")
            output_ensemble = gr.Textbox(label="Ensemble Prediction")
            output_modelwise = gr.Textbox(label="Model-wise Predictions")
            output_error = gr.Textbox(label="Error Log")
            analyze_btn = gr.Button("Analyze")
            analyze_btn.click(analyze, inputs=input_text, outputs=[output_ensemble, output_modelwise, output_error])

        with gr.TabItem("Reddit Scraper"):
            scraped_text = gr.Textbox(label="Scraped Reddit Post")
            scraped_ensemble = gr.Textbox(label="Ensemble Prediction")
            scraped_modelwise = gr.Textbox(label="Model-wise Predictions")
            scraped_error = gr.Textbox(label="Error Log")
            scrape_btn = gr.Button("Scrape and Analyze")
            scrape_btn.click(scrape_and_predict, inputs=[], outputs=[scraped_text, scraped_ensemble, scraped_modelwise, scraped_error])

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e0dc8b5b6a5d8ef5b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
